In [1]:
import os
os.chdir('..')

os.environ["DJANGO_SETTINGS_MODULE"] = "course_management.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ["IS_LOCAL"] = "1"

import django
django.setup()

IS_LOCAL=True
['C:\\Users\\alexe\\.virtualenvs\\course-management-platform-wiAsnpQu\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\alexe\\AppData\\Roaming\\jupyter\\runtime\\kernel-9fe39702-c5c9-4508-8565-76ea437bff09.json']
Is test: False


In [2]:
from django.contrib.auth import get_user_model # noqa: E402

from courses.models import (  # noqa: E402
    Course,
    Enrollment,
    Homework,
    Question,
    Submission,
    Answer,
    QuestionTypes,
    AnswerTypes,
    Project,
    ProjectSubmission,
    QUESTION_ANSWER_DELIMITER,
)


User = get_user_model()

course = Course.objects.get(slug="fake-course")


In [3]:
from courses.projects import assign_peer_reviews_for_project, score_project

from courses.models import ReviewCriteria, ReviewCriteriaTypes, PeerReview, PeerReviewState, CriteriaResponse

In [4]:
import random
from django.utils import timezone


In [5]:
criteria = ReviewCriteria.objects.filter(course=course)
enrollments = Enrollment.objects.filter(course=course)

In [6]:
p1 = Project.objects.get(
    course=course,
    slug=f"project-1",
)

p2 = Project.objects.get(
    course=course,
    slug=f"project-2",
)

In [ ]:
p = p2

assign_peer_reviews_for_project(p)

submissions = ProjectSubmission.objects.filter(enrollment__in=enrollments, project=p)
reviews = PeerReview.objects.filter(reviewer__in=submissions)

for r in reviews:
    if random.uniform(0, 1) < 0.2:
        continue

    for c in criteria:
        options = c.options
        
        if c.review_criteria_type == ReviewCriteriaTypes.RADIO_BUTTONS.value:
            i = random.randint(0, len(options) - 1)
            answer = str(i + 1)
        if c.review_criteria_type == ReviewCriteriaTypes.CHECKBOXES.value:
            answers = []
            for i in range(len(options)):
                if random.uniform(0, 1) < 0.3:
                    answers.append(str(i + 1))
            answer = ",".join(answers)

        CriteriaResponse.objects.create(
            review=r,
            criteria=c,
            answer=answer,
        )

    r.state = PeerReviewState.SUBMITTED.value
    r.save()

In [7]:
p1.peer_review_due_date = timezone.now()
score_project(p1)

INFO updading 21 submissions...
INFO Updating leaderboard for course 1
INFO Updated leaderboard in 0.02 seconds
INFO Project 2 scored in 0.37 seconds.


(<ProjectActionStatus.OK: 'OK'>,
 "Project 2 scored and state updated to 'COMPLETED'.")